In [2]:
import torch
from torch import nn
import numpy as np
import pandas

In [515]:
features_path = "features.csv"
labels_path = "labels.csv"
dataset_path = "data.csv"

In [443]:
class Network(nn.Module):
    def __init__(self):
        super().__init__()

        # Inputs to hidden layer linear transformation
        self.input = nn.Linear(3, 100)
        self.first_hidden = nn.Linear(100, 100)
        self.second_hidden = nn.Linear(100, 100)
        self.third_hidden = nn.Linear(100, 100)
        self.output = nn.Linear(100, 2)

        self.relu = nn.ReLU()

    def forward(self, x:[])->[]:
        # 3 -> 2
        x = self.input(x)
        x = self.relu(x)
        # 5 -> 5
        x = self.first_hidden(x)
        x = self.relu(x)
        # 5 -> 5
        x = self.second_hidden(x)
        x = self.relu(x)
        # 5 -> 5
        x = self.third_hidden(x)
        x = self.relu(x)
        # 2 -> 2
        x = self.output(x)

        return x

In [595]:
class Dataset():
    
    
    def get_dataset(path_to_dataset:str)->pandas.DataFrame:
        return pandas.read_csv(path_to_dataset, header=None)
    
    
    def get_dataset_no_zeroes(dataframe:pandas.DataFrame)->pandas.DataFrame:
        return dataframe[dataframe[0] != 0]
    
    
    def get_features(dataframe:pandas.DataFrame)->torch.Tensor:
        return torch.Tensor(dataframe.iloc[:,[0,1,2]].values)
    
    
    def get_labels(dataframe:pandas.DataFrame)->torch.Tensor:
        return torch.Tensor(dataframe.iloc[:,[3,4]].values)
    
    
    def get_features_no_zeroes(dataframe:pandas.DataFrame)->torch.Tensor:
        return torch.tensor(dataframe[dataframe[0] != 0].iloc[:,[0,1,2]].values)
    
    
    def get_labels_no_zeroes(dataframe:pandas.DataFrame)->torch.Tensor:
        return torch.tensor(dataframe[dataframe[0] != 0].iloc[:,[3, 4]].values)
    

def transform_features(tensor:torch.tensor)->torch.tensor:
    return np.log10(tensor)


def transform_labels(tensor:torch.tensor)->torch.tensor:
    tensor[:,0] = np.log10(tensor[:,0])
    tensor[:,1] = np.log10(-1*tensor[:,1])
    return tensor


def norm(tensor:torch.tensor)->torch.tensor:
    return torch.nn.functional.normalize(tensor).float()
    
    
def magnitude(vector:np.array)->float:
    return np.linalg.norm(vector)
    

In [581]:
# Get the dataset from PATH
dataset = Dataset.get_dataset_no_zeroes(Dataset.get_dataset(dataset_path))
dataset

,0,1,2,3,4
0,2.200000e-07,5.000000e-07,5.714324e-02,0.000002,-0.019457
1,3.140000e-08,8.000000e-07,7.142883e-02,0.000002,-0.000399
2,1.890000e-07,8.000000e-07,9.000000e-07,0.000002,-0.003692
3,3.140000e-08,6.710000e-07,7.142883e-02,0.000002,-0.016767
4,6.290000e-08,7.140000e-07,2.857207e-02,0.000002,-0.008122
...,...,...,...,...,...
507,1.570000e-07,6.290000e-07,2.857207e-02,0.000002,-0.010203
508,1.570000e-07,5.430000e-07,2.857207e-02,0.000002,-0.009285
509,1.260000e-07,5.000000e-07,5.714324e-02,0.000002,-0.005189
510,3.140000e-08,7.570000e-07,2.857207e-02,0.000002,-0.000702


In [664]:
# GET THE TRAINING SET
training_set = dataset.sample(frac = 0.85)
training_set

,0,1,2,3,4
211,1.890000e-07,5.860000e-07,1.000000e-01,0.000002,-0.004594
490,6.290000e-08,6.710000e-07,9.000000e-07,0.000002,-0.002086
24,6.290000e-08,7.140000e-07,8.571441e-02,0.000002,-0.035842
255,1.570000e-07,5.860000e-07,7.142883e-02,0.000002,-0.003743
160,9.430000e-08,6.290000e-07,4.285766e-02,0.000002,-0.006933
...,...,...,...,...,...
21,1.570000e-07,5.860000e-07,1.428649e-02,0.000002,-0.010930
18,6.290000e-08,7.140000e-07,1.428649e-02,0.000002,-0.002851
317,2.200000e-07,7.570000e-07,5.714324e-02,0.000002,-0.009943
325,1.570000e-07,5.430000e-07,8.571441e-02,0.000002,-0.005662


In [665]:
X = transform_features(Dataset.get_features(training_set))
X_normed = norm(X)
y = transform_labels(Dataset.get_labels(training_set))

In [666]:
testing_set = dataset.drop(training_set.index)
testing_set

,0,1,2,3,4
0,2.200000e-07,5.000000e-07,5.714324e-02,0.000002,-0.019457
3,3.140000e-08,6.710000e-07,7.142883e-02,0.000002,-0.016767
4,6.290000e-08,7.140000e-07,2.857207e-02,0.000002,-0.008122
15,2.200000e-07,5.430000e-07,5.714324e-02,0.000002,-0.005241
17,1.260000e-07,7.140000e-07,1.428649e-02,0.000002,-0.009594
...,...,...,...,...,...
456,1.260000e-07,6.710000e-07,4.285766e-02,0.000002,-0.008698
468,3.140000e-08,7.570000e-07,8.571441e-02,0.000002,-0.000279
475,9.430000e-08,6.290000e-07,2.857207e-02,0.000002,-0.005730
486,6.290000e-08,6.290000e-07,8.571441e-02,0.000002,-0.053611


In [668]:
X_test = transform_features(Dataset.get_features(testing_set))
X_test_normed = norm(X_test)
y_test = transform_labels(Dataset.get_labels(testing_set))

In [ ]:
gratingCouplerNet = Network()
optimizer = torch.optim.Adam(gratingCouplerNet.parameters(), lr=0.01)
loss_function = torch.nn.MSELoss()
epoch = 0
loss = 100
validation_loss = 100

while loss > 0.05:
    prediction = gratingCouplerNet(X_normed)
    loss = loss_function(prediction, y.float())
    test_prediction = gratingCouplerNet(X_test_normed)
    validation_loss = loss_function(test_prediction, y_test.float())
    
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    epoch += 1
    print("Epoch: {}, Training Loss: {:0.6f}, Validation Loss: {:0.6f}".format(epoch, loss, validation_loss))

Epoch: 1, Training Loss: 19.474678, Validation Loss: 19.724630
Epoch: 2, Training Loss: 17.623346, Validation Loss: 17.851967
Epoch: 3, Training Loss: 14.114474, Validation Loss: 14.295656
Epoch: 4, Training Loss: 7.704883, Validation Loss: 7.766698
Epoch: 5, Training Loss: 2.456374, Validation Loss: 2.258070
Epoch: 6, Training Loss: 8.417977, Validation Loss: 8.053440
Epoch: 7, Training Loss: 2.153508, Validation Loss: 1.963870
Epoch: 8, Training Loss: 0.263619, Validation Loss: 0.246949
Epoch: 9, Training Loss: 1.672676, Validation Loss: 1.763638
Epoch: 10, Training Loss: 2.884992, Validation Loss: 3.034387
Epoch: 11, Training Loss: 3.122385, Validation Loss: 3.301654
Epoch: 12, Training Loss: 2.613750, Validation Loss: 2.806322
Epoch: 13, Training Loss: 1.854544, Validation Loss: 2.046989
Epoch: 14, Training Loss: 1.479508, Validation Loss: 1.657253
Epoch: 15, Training Loss: 1.712294, Validation Loss: 1.859607
Epoch: 16, Training Loss: 1.770002, Validation Loss: 1.876900
Epoch: 17, 

Epoch: 177, Training Loss: 0.114677, Validation Loss: 0.109740
Epoch: 178, Training Loss: 0.114675, Validation Loss: 0.109730
Epoch: 179, Training Loss: 0.114673, Validation Loss: 0.109731
Epoch: 180, Training Loss: 0.114671, Validation Loss: 0.109740
Epoch: 181, Training Loss: 0.114669, Validation Loss: 0.109749
Epoch: 182, Training Loss: 0.114667, Validation Loss: 0.109754
Epoch: 183, Training Loss: 0.114665, Validation Loss: 0.109754
Epoch: 184, Training Loss: 0.114663, Validation Loss: 0.109754
Epoch: 185, Training Loss: 0.114661, Validation Loss: 0.109756
Epoch: 186, Training Loss: 0.114659, Validation Loss: 0.109762
Epoch: 187, Training Loss: 0.114657, Validation Loss: 0.109768
Epoch: 188, Training Loss: 0.114655, Validation Loss: 0.109770
Epoch: 189, Training Loss: 0.114653, Validation Loss: 0.109766
Epoch: 190, Training Loss: 0.114651, Validation Loss: 0.109757
Epoch: 191, Training Loss: 0.114649, Validation Loss: 0.109748
Epoch: 192, Training Loss: 0.114647, Validation Loss: 0

Epoch: 308, Training Loss: 0.114392, Validation Loss: 0.109379
Epoch: 309, Training Loss: 0.114390, Validation Loss: 0.109376
Epoch: 310, Training Loss: 0.114387, Validation Loss: 0.109373
Epoch: 311, Training Loss: 0.114385, Validation Loss: 0.109370
Epoch: 312, Training Loss: 0.114383, Validation Loss: 0.109366
Epoch: 313, Training Loss: 0.114380, Validation Loss: 0.109363
Epoch: 314, Training Loss: 0.114378, Validation Loss: 0.109360
Epoch: 315, Training Loss: 0.114376, Validation Loss: 0.109357
Epoch: 316, Training Loss: 0.114373, Validation Loss: 0.109354
Epoch: 317, Training Loss: 0.114371, Validation Loss: 0.109351
Epoch: 318, Training Loss: 0.114369, Validation Loss: 0.109348
Epoch: 319, Training Loss: 0.114366, Validation Loss: 0.109345
Epoch: 320, Training Loss: 0.114364, Validation Loss: 0.109342
Epoch: 321, Training Loss: 0.114362, Validation Loss: 0.109339
Epoch: 322, Training Loss: 0.114359, Validation Loss: 0.109336
Epoch: 323, Training Loss: 0.114357, Validation Loss: 0

Epoch: 464, Training Loss: 0.113994, Validation Loss: 0.108893
Epoch: 465, Training Loss: 0.113991, Validation Loss: 0.108890
Epoch: 466, Training Loss: 0.113988, Validation Loss: 0.108886
Epoch: 467, Training Loss: 0.113985, Validation Loss: 0.108883
Epoch: 468, Training Loss: 0.113982, Validation Loss: 0.108879
Epoch: 469, Training Loss: 0.113979, Validation Loss: 0.108876
Epoch: 470, Training Loss: 0.113976, Validation Loss: 0.108873
Epoch: 471, Training Loss: 0.113973, Validation Loss: 0.108869
Epoch: 472, Training Loss: 0.113970, Validation Loss: 0.108866
Epoch: 473, Training Loss: 0.113967, Validation Loss: 0.108862
Epoch: 474, Training Loss: 0.113964, Validation Loss: 0.108859
Epoch: 475, Training Loss: 0.113961, Validation Loss: 0.108855
Epoch: 476, Training Loss: 0.113958, Validation Loss: 0.108852
Epoch: 477, Training Loss: 0.113955, Validation Loss: 0.108848
Epoch: 478, Training Loss: 0.113952, Validation Loss: 0.108844
Epoch: 479, Training Loss: 0.113949, Validation Loss: 0

Epoch: 629, Training Loss: 0.113208, Validation Loss: 0.108168
Epoch: 630, Training Loss: 0.113199, Validation Loss: 0.108159
Epoch: 631, Training Loss: 0.113190, Validation Loss: 0.108150
Epoch: 632, Training Loss: 0.113181, Validation Loss: 0.108141
Epoch: 633, Training Loss: 0.113172, Validation Loss: 0.108132
Epoch: 634, Training Loss: 0.113162, Validation Loss: 0.108123
Epoch: 635, Training Loss: 0.113153, Validation Loss: 0.108114
Epoch: 636, Training Loss: 0.113143, Validation Loss: 0.108100
Epoch: 637, Training Loss: 0.113132, Validation Loss: 0.108083
Epoch: 638, Training Loss: 0.113120, Validation Loss: 0.108061
Epoch: 639, Training Loss: 0.113102, Validation Loss: 0.108027
Epoch: 640, Training Loss: 0.113081, Validation Loss: 0.107978
Epoch: 641, Training Loss: 0.113065, Validation Loss: 0.107925
Epoch: 642, Training Loss: 0.113052, Validation Loss: 0.107872
Epoch: 643, Training Loss: 0.113032, Validation Loss: 0.107806
Epoch: 644, Training Loss: 0.113015, Validation Loss: 0

Epoch: 780, Training Loss: 0.109715, Validation Loss: 0.105023
Epoch: 781, Training Loss: 0.109687, Validation Loss: 0.105009
Epoch: 782, Training Loss: 0.109660, Validation Loss: 0.104911
Epoch: 783, Training Loss: 0.109632, Validation Loss: 0.104910
Epoch: 784, Training Loss: 0.109607, Validation Loss: 0.104956
Epoch: 785, Training Loss: 0.109579, Validation Loss: 0.104897
Epoch: 786, Training Loss: 0.109556, Validation Loss: 0.104832
Epoch: 787, Training Loss: 0.109838, Validation Loss: 0.107050
Epoch: 788, Training Loss: 0.110062, Validation Loss: 0.104332
Epoch: 789, Training Loss: 0.109547, Validation Loss: 0.104499
Epoch: 790, Training Loss: 0.109837, Validation Loss: 0.105145
Epoch: 791, Training Loss: 0.109572, Validation Loss: 0.104088
Epoch: 792, Training Loss: 0.109616, Validation Loss: 0.104317
Epoch: 793, Training Loss: 0.109581, Validation Loss: 0.105458
Epoch: 794, Training Loss: 0.109465, Validation Loss: 0.105462
Epoch: 795, Training Loss: 0.109577, Validation Loss: 0

Epoch: 918, Training Loss: 0.108551, Validation Loss: 0.104641
Epoch: 919, Training Loss: 0.108545, Validation Loss: 0.104639
Epoch: 920, Training Loss: 0.108539, Validation Loss: 0.104641
Epoch: 921, Training Loss: 0.108533, Validation Loss: 0.104635
Epoch: 922, Training Loss: 0.108528, Validation Loss: 0.104618
Epoch: 923, Training Loss: 0.108522, Validation Loss: 0.104611
Epoch: 924, Training Loss: 0.108516, Validation Loss: 0.104610
Epoch: 925, Training Loss: 0.108510, Validation Loss: 0.104604
Epoch: 926, Training Loss: 0.108504, Validation Loss: 0.104600
Epoch: 927, Training Loss: 0.108498, Validation Loss: 0.104597
Epoch: 928, Training Loss: 0.108492, Validation Loss: 0.104585
Epoch: 929, Training Loss: 0.108486, Validation Loss: 0.104582
Epoch: 930, Training Loss: 0.108480, Validation Loss: 0.104583
Epoch: 931, Training Loss: 0.108474, Validation Loss: 0.104580
Epoch: 932, Training Loss: 0.108468, Validation Loss: 0.104583
Epoch: 933, Training Loss: 0.108462, Validation Loss: 0

Epoch: 1051, Training Loss: 0.107342, Validation Loss: 0.103648
Epoch: 1052, Training Loss: 0.107328, Validation Loss: 0.103630
Epoch: 1053, Training Loss: 0.107313, Validation Loss: 0.103593
Epoch: 1054, Training Loss: 0.107298, Validation Loss: 0.103564
Epoch: 1055, Training Loss: 0.107282, Validation Loss: 0.103561
Epoch: 1056, Training Loss: 0.107268, Validation Loss: 0.103559
Epoch: 1057, Training Loss: 0.107255, Validation Loss: 0.103576
Epoch: 1058, Training Loss: 0.107242, Validation Loss: 0.103602
Epoch: 1059, Training Loss: 0.107229, Validation Loss: 0.103635
Epoch: 1060, Training Loss: 0.107216, Validation Loss: 0.103656
Epoch: 1061, Training Loss: 0.107202, Validation Loss: 0.103656
Epoch: 1062, Training Loss: 0.107187, Validation Loss: 0.103617
Epoch: 1063, Training Loss: 0.107172, Validation Loss: 0.103557
Epoch: 1064, Training Loss: 0.107157, Validation Loss: 0.103509
Epoch: 1065, Training Loss: 0.107141, Validation Loss: 0.103476
Epoch: 1066, Training Loss: 0.107125, Va

Epoch: 1202, Training Loss: 0.103271, Validation Loss: 0.096685
Epoch: 1203, Training Loss: 0.103227, Validation Loss: 0.096992
Epoch: 1204, Training Loss: 0.103134, Validation Loss: 0.096563
Epoch: 1205, Training Loss: 0.103103, Validation Loss: 0.096197
Epoch: 1206, Training Loss: 0.103051, Validation Loss: 0.096612
Epoch: 1207, Training Loss: 0.102953, Validation Loss: 0.096121
Epoch: 1208, Training Loss: 0.102926, Validation Loss: 0.095795
Epoch: 1209, Training Loss: 0.102851, Validation Loss: 0.096217
Epoch: 1210, Training Loss: 0.102776, Validation Loss: 0.095829
Epoch: 1211, Training Loss: 0.102722, Validation Loss: 0.095388
Epoch: 1212, Training Loss: 0.102628, Validation Loss: 0.095439
Epoch: 1213, Training Loss: 0.102604, Validation Loss: 0.095539
Epoch: 1214, Training Loss: 0.102491, Validation Loss: 0.095118
Epoch: 1215, Training Loss: 0.102475, Validation Loss: 0.094613
Epoch: 1216, Training Loss: 0.102383, Validation Loss: 0.095091
Epoch: 1217, Training Loss: 0.102334, Va

In [695]:
X_test[0]

tensor([-6.6576, -6.3010, -1.2430])

In [696]:
prediction = gratingCouplerNet(X_test_normed)

In [697]:
loss_function(prediction, y_test.float())

tensor(0.0402, grad_fn=<MseLossBackward>)

In [700]:
prediction[0]

tensor([-5.8148, -2.2623], grad_fn=<SelectBackward>)

In [701]:
y_test[0]

tensor([-5.8125, -1.7109])

In [648]:
-1*np.power(10, y_test[0][1].detach().numpy())

-0.01945726046799541

In [651]:
-1*np.power(10, gratingCouplerNet(X_test_normed[0]).detach().numpy()[1])

-0.005269782584722879

In [654]:
-1*np.power(10, y[0][1].detach().numpy())

-0.005151365999223099

In [655]:
-1*np.power(10, gratingCouplerNet(X_normed[0]).detach().numpy()[1])

-0.008357428100215211